# produce the figures/numbers for the cosmic rejection paper

In [1]:
import sys; sys.path.insert(0, '../'); sys.path.insert(0, '../analysis_notes/mupClassification/'); 
from ccqe_notebook_tools import * 
from mupClassification_notebooks import *
import seaborn as sns;sns.set(style="white", color_codes=True ,font_scale=2);
%matplotlib inline

In [2]:
# ------------------------------------------------
# Aug-30, 2017
def get_Nreduced_MCbnbDATAcosmic(reduced_MCbnbDATAcosmic = dict()):
    Noriginal , Nreduced , freduced = dict() , dict() , dict()
    for pair_type in pair_types:
        sam = MCbnbDATAcosmicSamples[pair_type]
        Noriginal[pair_type] = len(MCbnbDATAcosmicSamples[pair_type])
        Nreduced[pair_type] = float(len(reduced_MCbnbDATAcosmic[pair_type]))
        freduced[pair_type] = 100.0 * Nreduced[pair_type]/Noriginal[pair_type]
    return Nreduced , freduced
# ------------------------------------------------

      
# ------------------------------------------------
# Oct. 13, 2017
def get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=None, cut_name = 'PIDa', cut_label=None , reduced_MCbnbDATAcosmic = dict()):
    ''' 
        return
        eff (mu-p) , pur (mu-p), eff (CC 1p 0pi) , pur (CC 1p 0pi)
    '''
    eff = dict()
    pur = dict()
    Nreduced_MCbnbDATAcosmic , freduced_MCbnbDATAcosmic = get_Nreduced_MCbnbDATAcosmic(reduced_MCbnbDATAcosmic=reduced_MCbnbDATAcosmic)
    Ntot = (Nreduced_MCbnbDATAcosmic['1mu-1p']+Nreduced_MCbnbDATAcosmic['cosmic']+Nreduced_MCbnbDATAcosmic['other pairs'])
    
    eff['1mu-1p'] = freduced_MCbnbDATAcosmic['1mu-1p']
    pur['1mu-1p'] = 100.*Nreduced_MCbnbDATAcosmic['1mu-1p']/Ntot if Ntot>0 else 0
    
    eff['CC 1p 0pi'] = freduced_MCbnbDATAcosmic['CC 1p 0pi']
    pur['CC 1p 0pi'] = 100.*Nreduced_MCbnbDATAcosmic['CC 1p 0pi']/Ntot if Ntot>0 else 0
    
    pureff_MCbnbDATAcosmic_cut = pd.DataFrame({'label':cut_label
                               ,'$\mu p$ eff.':'%.1f'%eff['1mu-1p']+'%'
                               ,'$\mu p$ pur.':'%.1f'%pur['1mu-1p']+'%'
                               ,'CC$0\pi 1 p$ eff.':'%.1f'%freduced_MCbnbDATAcosmic['CC 1p 0pi']+'%'
                               ,'CC$0\pi 1 p$ pur.':'%.1f'%(100.*Nreduced_MCbnbDATAcosmic['CC 1p 0pi']/Ntot if Ntot>0 else 0)+'%'}
                               , index=[cut_name]
                              )
    for pair_type in pair_types: pureff_MCbnbDATAcosmic_cut[pair_type] = '%.1f'%freduced_MCbnbDATAcosmic[pair_type]+'%' +' (%.0f)'%Nreduced_MCbnbDATAcosmic[pair_type]
    pureff_MCbnbDATAcosmic = pureff_MCbnbDATAcosmic.append(pureff_MCbnbDATAcosmic_cut)
    return pureff_MCbnbDATAcosmic
# ------------------------------------------------

In [3]:
MCbnbDATAcosmicPairsFV, MCbnbDATAcosmicSamples = load_MCbnbDATAcosmicSamples()

28757 pairs from MC-BNB + cosmic DATA overlay
18979 pairs in FV
5094 are 1mu-1p, 26.8%
4360 are other pairs, 23.0%
9525 are cosmic, 50.2%
--------------------------------------------------------------
2518 are CC 1p 0pi, 13.3%


# apply the cuts in order to
    1 select the optimal ones, and
    2 study the cut sensitivity

In [4]:
pair_types   = ['1mu-1p'   ,'other pairs','cosmic' ,'CC 1p 0pi'    ]
MClabels     = ['1$\\mu$1p','other pairs','cosmic' ,'CC 1p 0$\\pi$']
MCcolors     = ['teal'     ,'red'        ,'orange' ,'blue'         ]
MCcmaps      = ['Greens'   ,'Reds'       ,'Oranges','Blues'        ]

In [6]:
def check_cuts(PIDa_p_min=13
    ,minPEcut = 100
    ,maxdYZcut = 200                              
    ,delta_theta_12=55  # deg.
    ,opt_box=(50,100) # [Nwires x Nticks]
    ,r_max_RdQ_CC1p0pi = 0.35 # sphere in U,V,Y space, apply a cut only to CC1p0pi
    # --- -- --- - -- -- --- --
    ,delta_Delta_phi=35 # deg.
    ,Pt_max=0.35        # GeV/c
    ):
    
    reduced_MCbnbDATAcosmic = dict()
    pureff_MCbnbDATAcosmic = pd.DataFrame()
    pureff_MCbnbDATAcosmic_numbers = pd.DataFrame()

    for pair_type in pair_types:#{
        reduced_MCbnbDATAcosmic[pair_type] = MCbnbDATAcosmicSamples[pair_type]
    #}
    reduced_MCbnbDATAcosmicSamples['no cut'] = reduced_MCbnbDATAcosmic

    # before cuts
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam
    #}
    reduced_MCbnbDATAcosmicSamples['no cut'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name = 'no cut', cut_label='no cut', reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


    # cut 1: PIDa
    # -- -- -- -- -- ---- - ---- 
    cut_name , cut_label = 'PIDa','${PID}_a>%.0f$'%PIDa_p_min
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[sam['PIDa_assigned_proton']>PIDa_p_min]
    #}
    reduced_MCbnbDATAcosmicSamples['PIDa'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


    # cut 2: Optical filtering
    # -- -- -- -- -- ---- - ---- 
    cut_name , cut_label = 'flash', 'optical filter'
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['PIDa'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['ClosestFlash_TotalPE'] > minPEcut)
                                                     &(sam['ClosestFlash_YZdistance'] < maxdYZcut)]
    #}
    reduced_MCbnbDATAcosmicSamples['flash'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name=cut_name,cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


    # cut 3: require that the longer track is the one with larger PIDa
    # -- -- -- -- -- ---- - ---- 
    cut_name , cut_label = 'length', r'$l_{\mu}>l_{p}$'
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['flash'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[sam['PIDa_long'] < sam['PIDa_short']]
    #}
    reduced_MCbnbDATAcosmicSamples['length'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)

    # cut 4: theta_{12}
    # -- -- -- -- -- ---- - ---- 
    cut_name, cut_label='non-collinearity' ,'$|\theta_{1,2}-90^0|<%.0f^0$'%delta_theta_12
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['length'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[np.abs(sam['theta_12']-90)<delta_theta_12]
    #}
    reduced_MCbnbDATAcosmicSamples['non-collinearity'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


    # cut 5: vertex activity (RdQ)
    # -- -- -- -- -- ---- - ---- 
    cut_name , cut_label = 'vertex activity' , '$\sqrt{\sum_{p=0,1,2}(R_{\Delta Q}^{p}-1)^2}<%.2f$'%r_max_RdQ_CC1p0pi
    box_str='[%d wires x %d ticks]'%(opt_box[0],opt_box[1])
    Ru,Rv,Ry = 'RdQaroundVertex[plane 0]'+box_str,'RdQaroundVertex[plane 1]'+box_str,'RdQaroundVertex[plane 2]'+box_str
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{         
        sam = reduced_MCbnbDATAcosmicSamples['non-collinearity'][pair_type]
        sam = sam[(sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1) 
                      | 
                      (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) <= r_max_RdQ_CC1p0pi) ]
        reduced_MCbnbDATAcosmic[pair_type] = sam        
    #}
    reduced_MCbnbDATAcosmicSamples['vertex activity'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name=cut_name ,cut_label=cut_label , reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)

    # cut 6: delta phi
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['vertex activity'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[np.abs(sam['delta_phi']-180.)<delta_Delta_phi]
    #}
    reduced_MCbnbDATAcosmicSamples['delta phi'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name = 'delta phi', cut_label='$|\Delta \phi - \pi|<%.0f^0$'%delta_Delta_phi, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)

    # cut 6.5: Pt<0.35 without application of the $\Delta phi$ cut
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['vertex activity'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[sam['reco_Pt']<Pt_max]
    #}
    reduced_MCbnbDATAcosmicSamples['Pt No Delta phi'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name='Pt no Delta phi', cut_label='$p_{t}<%.2f$ GeV/c'%Pt_max, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


    # cut 7: Pt<0.35
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['delta phi'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[sam['reco_Pt']<Pt_max]
    #}
    reduced_MCbnbDATAcosmicSamples['soft Pt'] = reduced_MCbnbDATAcosmic
    pureff_MCbnbDATAcosmic = get_pureff_MCbnbDATAcosmic_cut(pureff_MCbnbDATAcosmic=pureff_MCbnbDATAcosmic,cut_name='soft Pt', cut_label='$p_{t}<%.2f$ GeV/c'%Pt_max, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)

    return pureff_MCbnbDATAcosmic

In [8]:
pureff_MCbnbDATAcosmic = check_cuts()
pureff_MCbnbDATAcosmic

,$\mu p$ eff.,$\mu p$ pur.,CC$0\pi 1 p$ eff.,CC$0\pi 1 p$ pur.,label,1mu-1p,other pairs,cosmic,CC 1p 0pi
no cut,100.0%,26.8%,100.0%,13.3%,no cut,100.0% (5094),100.0% (4360),100.0% (9525),100.0% (2518)
PIDa,85.5%,53.6%,84.6%,26.2%,${PID}_a>13$,85.5% (4354),55.3% (2409),14.3% (1364),84.6% (2131)
flash,80.4%,57.9%,79.5%,28.3%,optical filter,80.4% (4096),53.2% (2318),6.9% (661),79.5% (2002)
length,66.6%,67.3%,69.5%,34.7%,$l_{\mu}>l_{p}$,66.6% (3393),29.3% (1279),3.9% (371),69.5% (1751)
non-collinearity,59.8%,73.2%,64.8%,39.2%,"$|\theta_{1,2}-90^0|<55^0$",59.8% (3044),20.8% (909),2.1% (203),64.8% (1631)
vertex activity,40.9%,76.9%,55.8%,51.9%,"$\sqrt{\sum_{p=0,1,2}(R_{\Delta Q}^{p}-1)^2}<0...",40.9% (2081),11.8% (516),1.1% (108),55.8% (1404)
delta phi,27.2%,90.8%,42.8%,70.5%,$|\Delta \phi - \pi|<35^0$,27.2% (1387),2.6% (115),0.3% (25),42.8% (1077)
Pt no Delta phi,31.0%,85.0%,46.0%,62.4%,$p_{t}<0.35$ GeV/c,31.0% (1578),5.3% (230),0.5% (49),46.0% (1159)
soft Pt,24.0%,93.1%,38.5%,73.8%,$p_{t}<0.35$ GeV/c,24.0% (1223),1.7% (74),0.2% (16),38.5% (969)


In [9]:
pureff_MCbnbDATAcosmic = check_cuts(PIDa_p_min=12
    ,minPEcut = 100
    ,maxdYZcut = 75                              
    ,delta_theta_12=55  # deg.
    ,r_max_RdQ_CC1p0pi = 0.33 # sphere in U,V,Y space, apply a cut only to CC1p0pi
    # --- -- --- - -- -- --- --
    ,delta_Delta_phi=40 # deg.
    ,Pt_max=0.35        # GeV/c
    )
pureff_MCbnbDATAcosmic

,$\mu p$ eff.,$\mu p$ pur.,CC$0\pi 1 p$ eff.,CC$0\pi 1 p$ pur.,label,1mu-1p,other pairs,cosmic,CC 1p 0pi
no cut,100.0%,26.8%,100.0%,13.3%,no cut,100.0% (5094),100.0% (4360),100.0% (9525),100.0% (2518)
PIDa,88.1%,52.2%,87.1%,25.5%,${PID}_a>12$,88.1% (4487),57.9% (2526),16.6% (1584),87.1% (2194)
flash,45.8%,58.2%,45.7%,28.7%,optical filter,45.8% (2334),33.3% (1453),2.4% (224),45.7% (1150)
length,37.7%,66.9%,40.1%,35.2%,$l_{\mu}>l_{p}$,37.7% (1920),18.9% (822),1.3% (128),40.1% (1010)
non-collinearity,33.9%,72.4%,37.1%,39.1%,"$|\theta_{1,2}-90^0|<55^0$",33.9% (1729),13.7% (596),0.7% (64),37.1% (934)
vertex activity,23.5%,76.1%,32.8%,52.5%,"$\sqrt{\sum_{p=0,1,2}(R_{\Delta Q}^{p}-1)^2}<0...",23.5% (1195),7.8% (339),0.4% (37),32.8% (825)
delta phi,15.8%,89.6%,25.0%,69.9%,$|\Delta \phi - \pi|<40^0$,15.8% (806),1.9% (84),0.1% (10),25.0% (629)
Pt no Delta phi,17.4%,84.0%,26.6%,63.6%,$p_{t}<0.35$ GeV/c,17.4% (885),3.6% (156),0.1% (13),26.6% (670)
soft Pt,13.6%,91.8%,22.2%,74.3%,$p_{t}<0.35$ GeV/c,13.6% (692),1.3% (57),0.1% (5),22.2% (560)


# sensitivity to the cuts using a Gaussian sampling

In [11]:
Nrand = 1000
opt_box=(50,100)
box_str='[%d wires x %d ticks]'%(opt_box[0],opt_box[1])
Ru,Rv,Ry = 'RdQaroundVertex[plane 0]'+box_str,'RdQaroundVertex[plane 1]'+box_str,'RdQaroundVertex[plane 2]'+box_str

Ncosmic_det,fcosmic_det,NCC1p0pi_det,fCC1p0pi_det=[],[],[],[]
Ncosmic_all,fcosmic_all,NCC1p0pi_all,fCC1p0pi_all=[],[],[],[]

for irand in range(Nrand):
    if irand%(Nrand/10)==0: print '%.1f'%(100.*float(irand)/Nrand)+'%'
    PIDa_p_min = np.random.normal( 12 , 1)
    minPEcut = np.random.normal( 100 , 20)
    maxdYZcut = np.random.normal( 200 , 50)
    delta_theta_12 = np.random.normal( 60 , 5 )
    r_max_RdQ_CC1p0pi = np.random.normal( 0.35 , 0.05 )
    delta_Delta_phi = np.random.normal( 40 , 5 )
    Pt_max = np.random.normal( 0.35 , 0.05 )
    
    # detector cuts
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut)
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi))]
    #}
    Ncosmic_det.append(len(reduced_MCbnbDATAcosmic['cosmic']))
    fcosmic_det.append(100.*float(Ncosmic_det[-1])/len(reduced_MCbnbDATAcosmicSamples['no cut']['cosmic']))
    NCC1p0pi_det.append(len(reduced_MCbnbDATAcosmic['CC 1p 0pi']))
    fCC1p0pi_det.append(100.*float(NCC1p0pi_det[-1])/len(reduced_MCbnbDATAcosmicSamples['no cut']['CC 1p 0pi']))

    # all cuts
    reduced_MCbnbDATAcosmic = dict()
    for pair_type in pair_types:#{
        sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
        reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut)
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi))
                                             &(np.abs(sam['delta_phi']-180.)<delta_theta_12)
                                             &(sam['reco_Pt']<Pt_max)]
    #}
    Ncosmic_all.append(len(reduced_MCbnbDATAcosmic['cosmic']))
    fcosmic_all.append(100.*float(Ncosmic_all[-1])/len(reduced_MCbnbDATAcosmicSamples['no cut']['cosmic']))
    NCC1p0pi_all.append(len(reduced_MCbnbDATAcosmic['CC 1p 0pi']))
    fCC1p0pi_all.append(100.*float(NCC1p0pi_all[-1])/len(reduced_MCbnbDATAcosmicSamples['no cut']['CC 1p 0pi']))
            
Ncosmic,fcosmic,NCC1p0pi,fCC1p0pi = np.array(Ncosmic),np.array(fcosmic),np.array(NCC1p0pi),np.array(fCC1p0pi)
print 'done.'

0.0%
10.0%
20.0%
30.0%
40.0%
50.0%
60.0%
70.0%
80.0%
90.0%


NameError: name 'Ncosmic' is not defined

In [ ]:
fig = plt.figure(figsize=(16,12))

ax=fig.add_subplot(2,2,1)
plt.hist(Ncosmic_det,bins=np.linspace(0,201,51))
mean,std=np.mean(Ncosmic_det),np.std(Ncosmic_det)
set_axes(ax,r'$N_{cosmic}$',do_add_grid=True,title=r'$\mu=%.1f,\sigma=%.1f$'%(mean,std))

ax=fig.add_subplot(2,2,2)
plt.hist(NCC1p0pi_det,bins=100)
mean,std=np.mean(NCC1p0pi_det),np.std(NCC1p0pi_det)
set_axes(ax,r'$N_{CC1p0\pi}$',do_add_grid=True,title=r'$\mu=%.1f,\sigma=%.1f$'%(mean,std))

ax=fig.add_subplot(2,2,3)
plt.hist(fcosmic_det,bins=np.linspace(0,2,50))
mean,std=np.mean(fcosmic_det),np.std(fcosmic_det)
set_axes(ax,r'$\epsilon_{cosmic}$ [%]',do_add_grid=True,title=(r'$\mu=%.2f$'%mean+'%'+r'$,\sigma=%.2f$'%std+'%'))

ax=fig.add_subplot(2,2,4)
plt.hist(fCC1p0pi_det,bins=100)
mean,std=np.mean(fCC1p0pi_det),np.std(fCC1p0pi_det)
set_axes(ax,r'$\epsilon_{CC1p0\pi}$ [%]',do_add_grid=True,title=(r'$\mu=%.2f$'%mean+'%'+r'$,\sigma=%.2f$'%std+'%'))


plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(16,12))

ax=fig.add_subplot(2,2,1)
plt.hist(Ncosmic_all,bins=np.linspace(0,50,51))
mean,std=np.mean(Ncosmic_all),np.std(Ncosmic_all)
set_axes(ax,r'$N_{cosmic}$',do_add_grid=True,title=r'$\mu=%.1f,\sigma=%.1f$'%(mean,std))

ax=fig.add_subplot(2,2,2)
plt.hist(NCC1p0pi_det,bins=100)
mean,std=np.mean(NCC1p0pi_all),np.std(NCC1p0pi_all)
set_axes(ax,r'$N_{CC1p0\pi}$',do_add_grid=True,title=r'$\mu=%.1f,\sigma=%.1f$'%(mean,std))

ax=fig.add_subplot(2,2,3)
plt.hist(fcosmic_all,bins=np.linspace(0,0.51,50))
mean,std=np.mean(fcosmic_all),np.std(fcosmic_all)
set_axes(ax,r'$\epsilon_{cosmic}$ [%]',do_add_grid=True,title=(r'$\mu=%.2f$'%mean+'%'+r'$,\sigma=%.2f$'%std+'%'))

ax=fig.add_subplot(2,2,4)
plt.hist(fCC1p0pi_all,bins=100)
mean,std=np.mean(fCC1p0pi_all),np.std(fCC1p0pi_all)
set_axes(ax,r'$\epsilon_{CC1p0\pi}$ [%]',do_add_grid=True,title=(r'$\mu=%.2f$'%mean+'%'+r'$,\sigma=%.2f$'%std+'%'))


plt.tight_layout()

# Sensitivity when all (fixed) cut parameters are shifter together

In [ ]:
# cuts sensitivity
PIDa_p_min_minus = 11
PIDa_p_min_plus = 13
minPEcut_minus = 80
minPEcut_plus = 120
maxdYZcut_minus = 150
maxdYZcut_plus = 250
delta_theta_12_minus = 55 
delta_theta_12_plus = 65 
r_max_RdQ_CC1p0pi_minus = 0.3
r_max_RdQ_CC1p0pi_plus = 0.4
# --- -- --- - -- -- --- --
delta_Delta_phi_minus = 35
delta_Delta_phi_plus = 45
Pt_max_minus=0.3
Pt_max_plus=0.4
# --- -- --- - -- -- --- --

   
# -- -- - -- - -- - -- - -- - -- - -- - -- - --
# cut-sensitivity
# -- -- - -- - -- - -- - -- - -- - -- - -- - -- 
cut_name,cut_label  = 'detector cuts',r'detector cuts'
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    reduced_MCbnbDATAcosmic[pair_type] = reduced_MCbnbDATAcosmicSamples['vertex activity'][pair_type]
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


cut_name  = 'detector cuts all tight'
arguments = (PIDa_p_min_plus , minPEcut_plus , maxdYZcut_minus , delta_theta_12_minus , r_max_RdQ_CC1p0pi_minus)
cut_label = r'${PID}_a^p>%d, N_{PE}>%d, d_{YZ}<%d, |\theta_{1,2}-\frac{\pi}{2}|<%d^{0},r_{\Delta Q}<%.1f$'%arguments
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
    reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min_plus)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut_plus)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut_minus)
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12_minus)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi_minus))]
#}
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)
get_pureff_MCbnbDATAcosmic_numbers(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)

cut_name  = 'detector cuts all relaxed'
arguments = (PIDa_p_min_minus , minPEcut_minus , maxdYZcut_plus , delta_theta_12_plus , r_max_RdQ_CC1p0pi_plus)
cut_label = r'${PID}_a^p>%d, N_{PE}>%d, d_{YZ}<%d, |\theta_{1,2}-\frac{\pi}{2}|<%d^{0},r_{\Delta Q}<%.1f$'%arguments
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
    reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min_minus)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut_minus)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut_plus)    
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12_plus)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi_plus))]
#}
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic=reduced_MCbnbDATAcosmic)
get_pureff_MCbnbDATAcosmic_numbers(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic=reduced_MCbnbDATAcosmic)



cut_name,cut_label  = 'all cuts',r'all cuts'
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    reduced_MCbnbDATAcosmic[pair_type] = reduced_MCbnbDATAcosmicSamples['soft Pt'][pair_type]
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)


cut_name  = 'all cuts all tight'
arguments = (PIDa_p_min_plus , minPEcut_plus , maxdYZcut_minus , delta_theta_12_minus , r_max_RdQ_CC1p0pi_minus, delta_Delta_phi_minus,Pt_max_minus)
cut_label = r'all tight'
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
    reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min_plus)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut_plus)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut_minus)
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12_minus)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi_minus))
                                             &(np.abs(sam['delta_phi']-180.)<delta_Delta_phi_minus)
                                             &(sam['reco_Pt']<Pt_max_minus)]
#}
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)
get_pureff_MCbnbDATAcosmic_numbers(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)





cut_name  = 'all cuts all relaxed'
arguments = (PIDa_p_min_minus , minPEcut_minus , maxdYZcut_plus , delta_theta_12_plus , r_max_RdQ_CC1p0pi_plus, delta_Delta_phi_plus,Pt_max_plus)
cut_label = r'all relaxed'
reduced_MCbnbDATAcosmic = dict()
for pair_type in pair_types:#{
    sam = reduced_MCbnbDATAcosmicSamples['no cut'][pair_type]
    reduced_MCbnbDATAcosmic[pair_type] = sam[(sam['PIDa_assigned_proton']>PIDa_p_min_minus)
                                             &(sam['ClosestFlash_TotalPE'] > minPEcut_minus)                                                
                                             &(sam['ClosestFlash_YZdistance'] < maxdYZcut_plus)    
                                             &(sam['PIDa_long'] < sam['PIDa_short'])
                                             &(np.abs(sam['theta_12']-90)<delta_theta_12_plus)
                                             &((sam[Ru]==1) | (sam[Rv]==1) | (sam[Ry]==1)                                              
                                               | (np.sqrt( np.square(sam[Ru]-1) + np.square(sam[Rv]-1) + np.square(sam[Ry]-1) ) 
                                                <= r_max_RdQ_CC1p0pi_plus))
                                             &(np.abs(sam['delta_phi']-180.)<delta_Delta_phi_plus)
                                             &(sam['reco_Pt']<Pt_max_plus)]
#}
get_pureff_MCbnbDATAcosmic_cut(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)
get_pureff_MCbnbDATAcosmic_numbers(cut_name=cut_name, cut_label=cut_label, reduced_MCbnbDATAcosmic = reduced_MCbnbDATAcosmic)




